# Hosting Strands + LangGraph agent with Amazon Bedrock models in Amazon Bedrock AgentCore Runtime

## Overview

In this tutorial we will learn how to host your existing agent, using Amazon Bedrock AgentCore Runtime. 

We will focus on a LangGraph with Amazon Bedrock model example. For Strands Agents with Amazon Bedrock model check [here](../01-strands-with-bedrock-model)
and for a Strands Agents with an OpenAI model check [here](../03-strands-with-openai-model).

### Tutorial Details

| Information         | Details                                                                      |
|:--------------------|:-----------------------------------------------------------------------------|
| Tutorial type       | Conversational                                                               |
| Agent type          | Single                                                                       |
| Agentic Framework   | Strands + LangGraph                                                                    |
| LLM model           | Anthropic Claude Sonnet 3                                                    |
| Tutorial components | Hosting agent on AgentCore Runtime. Using Strands + LangGraph and Amazon Bedrock Model |
| Tutorial vertical   | Cross-vertical                                                               |
| Example complexity  | Easy                                                                         |
| SDK used            | Amazon BedrockAgentCore Python SDK and boto3                                 |

### Tutorial Architecture

In this tutorial we will describe how to deploy an existing agent to AgentCore runtime. 

For demonstration purposes, we will  use a LangGraph agent using Amazon Bedrock models

In our example we will use a very simple agent with two tools: `get_weather` and `get_time`. 

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### Tutorial Key Features

* Hosting Agents on Amazon Bedrock AgentCore Runtime
* Using Amazon Bedrock models
* Using LangGraph


## Prerequisites

To execute this tutorial you will need:
* Python 3.10+
* AWS credentials
* Amazon Bedrock AgentCore SDK
* LangGraph
* Docker running

In [ ]:
#!uv add -r requirements.txt --active

## Creating your agents and experimenting locally

Before we deploy our agents to AgentCore Runtime, let's develop and run them locally for experimentation purposes.

For production agentic applications we will need to decouple the agent creation process from the agent invocation one. With AgentCore Runtime, we will decorate the invocation part of our agent with the `@app.entrypoint` decorator and have it as the entry point for our runtime. Let's first look how each agent is developed during the experimentation phase.

The architecture here will look as following:

<div style="text-align:left">
    <img src="images/architecture_local.png" width="60%"/>
</div>

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
module_path = "../../../.."
sys.path.append(os.path.abspath(module_path))

In [4]:
from textwrap import dedent
from graph import build_graph
from src.utils.common_utils import get_message_from_string
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template

In [5]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Default level is INFO
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

def enable_debug_logging():
    """Enable debug level logging for more detailed execution information."""
    logging.getLogger(__name__).setLevel(logging.DEBUG)

# 로거 설정을 전역으로 한 번만 수행
logger = logging.getLogger(__name__)
logger.propagate = False
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
handler = logging.StreamHandler()
formatter = logging.Formatter('\n%(levelname)s [%(name)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)  # 기본 레벨은 INFO로 설정

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


In [6]:
# Create the graph
graph = build_graph()

In [7]:
def run_agent_workflow(user_input: str, debug: bool = False):
    """Run the agent workflow with the given user input.

    Args:
        user_input: The user's query or request
        debug: If True, enables debug level logging

    Returns:
        The final state after the workflow completes
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    if debug:
        enable_debug_logging()

    #logger.info(f"Starting workflow with user input: {user_input}")
    logger.info(f"{Colors.GREEN}===== Starting workflow ====={Colors.END}")
    logger.info(f"{Colors.GREEN}\nuser input: {user_input}{Colors.END}")
    
    user_prompts = dedent(
        '''
        Here is a user request: <user_request>{user_request}</user_request>
        '''
    )
    context = {"user_request": user_input}
    user_prompts = user_prompts.format(**context)
    messages = [get_message_from_string(role="user", string=user_prompts, imgs=[])]

        
    result = graph.invoke(
        input={
            # Runtime Variables
            "messages": messages,
            "request": user_input,
            "request_prompt": user_prompts
        },
        config={
            "recursion_limit": 100
        }
    )
    logger.debug(f"{Colors.RED}Final workflow state: {result}{Colors.END}")
    logger.info(f"{Colors.GREEN}===== Workflow completed successfully ====={Colors.END}")
    return result

In [8]:
import nest_asyncio
nest_asyncio.apply()

#remove_artifact_folder()

user_query = '''
    안녕 나는 장동진이라고 해. 만나서 반가워
    나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어. 
'''

result = run_agent_workflow(
    user_input=user_query,
    debug=False
)



INFO [__main__] ===== Starting workflow =====

INFO [__main__] 
user input: 
    안녕 나는 장동진이라고 해. 만나서 반가워
    나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어. 


INFO [graph.nodes] ===== Coordinator talking...... =====

INFO [src.utils.strands_sdk_utils] COORDINATOR - Prompt Cache Disabled


안녕하세요 장동진님! 저는 Bedrock-Manus입니다. 만나서 반갑습니다!

데이터 분석과 인사이트 추출에 대한 요청은 저희 플래너 시스템과 상담이 필요할 것 같습니다.

handoff_to_planner: I'll need to consult our planning system for this request.


INFO [graph.nodes] ===== Coordinator completed task =====

INFO [graph.nodes] ===== Planner generating full plan =====

INFO [src.utils.strands_sdk_utils] PLANNER - Prompt Cache Enabled


I need to analyze this user request that is in Korean. Let me translate and understand what the user is asking for:

"Hello, my name is Jang Dongjin. Nice to meet you.
I want to provide data and extract insights from it."

The user is introducing themselves as Jang Dongjin and they want to provide data from which they would like to extract insights. However, they haven't provided any specific data yet or details about what kind of insights they are looking for.

Let me create a plan for how we would approach this task once we receive the data:

1. First, we'll need the Coder to help with data processing and analysis once the user provides their data.
2. At the end, the Reporter will compile the insights into a comprehensive report.

Since there's no specific data provided yet, our first step should be to ask the user for the data and more details about what kinds of insights they're looking for. 

However, since I'm expected to create a plan based on the current information, I'll creat


INFO [graph.nodes] ===== Planner completed task =====

INFO [__main__] ===== Workflow completed successfully =====


In [ ]:
%%writefile strands_langgraph_bedrock.py

import logging
from textwrap import dedent
from graph import build_graph
from src.utils.common_utils import get_message_from_string
from src.utils.strands_sdk_utils import strands_utils
from src.prompts.template import apply_prompt_template

from bedrock_agentcore.runtime import BedrockAgentCoreApp

# Configure logging
logging.basicConfig(
    level=logging.INFO,  # Default level is INFO
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)

def enable_debug_logging():
    """Enable debug level logging for more detailed execution information."""
    logging.getLogger(__name__).setLevel(logging.DEBUG)

# 로거 설정을 전역으로 한 번만 수행
logger = logging.getLogger(__name__)
logger.propagate = False
for handler in logger.handlers[:]:
    logger.removeHandler(handler)
handler = logging.StreamHandler()
formatter = logging.Formatter('\n%(levelname)s [%(name)s] %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
logger.setLevel(logging.INFO)  # 기본 레벨은 INFO로 설정

class Colors:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'


# Create the graph
graph = build_graph()

def run_agent_workflow(user_input: str, debug: bool = False):
    """Run the agent workflow with the given user input.

    Args:
        user_input: The user's query or request
        debug: If True, enables debug level logging

    Returns:
        The final state after the workflow completes
    """
    if not user_input:
        raise ValueError("Input could not be empty")

    if debug:
        enable_debug_logging()

    #logger.info(f"Starting workflow with user input: {user_input}")
    logger.info(f"{Colors.GREEN}===== Starting workflow ====={Colors.END}")
    logger.info(f"{Colors.GREEN}\nuser input: {user_input}{Colors.END}")
    
    user_prompts = dedent(
        '''
        Here is a user request: <user_request>{user_request}</user_request>
        '''
    )
    context = {"user_request": user_input}
    user_prompts = user_prompts.format(**context)
    messages = [get_message_from_string(role="user", string=user_prompts, imgs=[])]

        
    result = graph.invoke(
        input={
            # Runtime Variables
            "messages": messages,
            "request": user_input,
            "request_prompt": user_prompts
        },
        config={
            "recursion_limit": 100
        }
    )
    logger.debug(f"{Colors.RED}Final workflow state: {result}{Colors.END}")
    logger.info(f"{Colors.GREEN}===== Workflow completed successfully ====={Colors.END}")
    return result

def strands_langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_langgraph_bedrock(json.loads(args.payload))
    print(response)



Writing strands_langgraph_bedrock.py


ModuleNotFoundError: No module named 'bedrock_agentcore'

In [ ]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
import argparse
import json
import operator
import math

# Create calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Calculate the result of a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # Define safe functions that can be used in expressions
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # Math functions
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # Basic operators (for explicit use)
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # Evaluate the expression safely
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# Create a custom weather tool
@tool
def weather():
    """Get weather"""  # Dummy implementation
    return "sunny"

# Define the agent using manual LangGraph construction
def create_agent():
    """Create and configure the LangGraph agent"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

def langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")


    import nest_asyncio
    nest_asyncio.apply()

#remove_artifact_folder()

user_query = '''
    안녕 나는 장동진이라고 해. 만나서 반가워
    나는 데이터를 제공하고 그것으로 부터 인사이트를 추출하고 싶어. 
'''

result = run_agent_workflow(
    user_input=user_query,
    debug=False
)


    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = langgraph_bedrock(json.loads(args.payload))
    print(response)

Writing langgraph_bedrock.py


#### Invoking local agent

In [28]:
!python langgraph_bedrock.py '{"prompt": "What is the weather now?"}'

INFO:     Started server process [538932]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8080 (Press CTRL+C to quit)
INFO:     Shutting down
^C
INFO:     Finished server process [538932]
ERROR:    Traceback (most recent call last):
  File "/usr/lib/python3.12/asyncio/runners.py", line 194, in run
    return runner.run(main)
           ^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/runners.py", line 118, in run
    return self._loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/base_events.py", line 674, in run_until_complete
    self.run_forever()
  File "/usr/lib/python3.12/asyncio/base_events.py", line 641, in run_forever
    self._run_once()
  File "/usr/lib/python3.12/asyncio/base_events.py", line 1987, in _run_once
    handle._run()
  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run
    self._context.run(self._callback,

## Preparing your agent for deployment on AgentCore Runtime

Let's now deploy our agents to AgentCore Runtime. To do so we need to:
* Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
* Initialize the App in our code with `app = BedrockAgentCoreApp()`
* Decorate the invocation function with the `@app.entrypoint` decorator
* Let AgentCoreRuntime control the running of the agent with `app.run()`

### Strands Agent SDK + LangGraph with Amazon Bedrock model
Let's start with our Strands Agent SDK + LangGraph using Amazon Bedrock model. Other examples with different frameworks and models are available in the parent directories

### Dockerfile


In [16]:
%%writefile langgraph_bedrock.py
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from bedrock_agentcore.runtime import BedrockAgentCoreApp
import argparse
import json
import operator
import math

app = BedrockAgentCoreApp()

# Create calculator tool
@tool
def calculator(expression: str) -> str:
    """
    Calculate the result of a mathematical expression.
    
    Args:
        expression: A mathematical expression as a string (e.g., "2 + 3 * 4", "sqrt(16)", "sin(pi/2)")
    
    Returns:
        The result of the calculation as a string
    """
    try:
        # Define safe functions that can be used in expressions
        safe_dict = {
            "__builtins__": {},
            "abs": abs, "round": round, "min": min, "max": max,
            "sum": sum, "pow": pow,
            # Math functions
            "sqrt": math.sqrt, "sin": math.sin, "cos": math.cos, "tan": math.tan,
            "log": math.log, "log10": math.log10, "exp": math.exp,
            "pi": math.pi, "e": math.e,
            "ceil": math.ceil, "floor": math.floor,
            "degrees": math.degrees, "radians": math.radians,
            # Basic operators (for explicit use)
            "add": operator.add, "sub": operator.sub,
            "mul": operator.mul, "truediv": operator.truediv,
        }
        
        # Evaluate the expression safely
        result = eval(expression, safe_dict)
        return str(result)
        
    except ZeroDivisionError:
        return "Error: Division by zero"
    except ValueError as e:
        return f"Error: Invalid value - {str(e)}"
    except SyntaxError:
        return "Error: Invalid mathematical expression"
    except Exception as e:
        return f"Error: {str(e)}"

# Create a custom weather tool
@tool
def weather():
    """Get weather"""  # Dummy implementation
    return "sunny"

# Define the agent using manual LangGraph construction
def create_agent():
    """Create and configure the LangGraph agent"""
    from langchain_aws import ChatBedrock
    
    # Initialize your LLM (adjust model and parameters as needed)
    llm = ChatBedrock(
        model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # or your preferred model
        model_kwargs={"temperature": 0.1}
    )
    
    # Bind tools to the LLM
    tools = [calculator, weather]
    llm_with_tools = llm.bind_tools(tools)
    
    # System message
    system_message = "You're a helpful assistant. You can do simple math calculation, and tell the weather."
    
    # Define the chatbot node
    def chatbot(state: MessagesState):
        # Add system message if not already present
        messages = state["messages"]
        if not messages or not isinstance(messages[0], SystemMessage):
            messages = [SystemMessage(content=system_message)] + messages
        
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Create the graph
    graph_builder = StateGraph(MessagesState)
    
    # Add nodes
    graph_builder.add_node("chatbot", chatbot)
    graph_builder.add_node("tools", ToolNode(tools))
    
    # Add edges
    graph_builder.add_conditional_edges(
        "chatbot",
        tools_condition,
    )
    graph_builder.add_edge("tools", "chatbot")
    
    # Set entry point
    graph_builder.set_entry_point("chatbot")
    
    # Compile the graph
    return graph_builder.compile()

# Initialize the agent
agent = create_agent()

@app.entrypoint
def langgraph_bedrock(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    
    # Create the input in the format expected by LangGraph
    response = agent.invoke({"messages": [HumanMessage(content=user_input)]})
    
    # Extract the final message content
    return response["messages"][-1].content

if __name__ == "__main__":
    app.run()

Overwriting langgraph_bedrock.py


## What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

* Creates an HTTP server that listens on the port 8080
* Implements the required `/invocations` endpoint for processing the agent's requirements
* Implements the `/ping` endpoint for health checks (very important for asynchronous agents)
* Handles proper content types and response formats
* Manages error handling according to the AWS standards

## Deploying the agent to AgentCore Runtime

The `CreateAgentRuntime` operation supports comprehensive configuration options, letting you specify container images, environment variables and encryption settings. You can also configure protocol settings (HTTP, MCP) and authorization mechanisms to control how your clients communicate with the agent. 

**Note:** Operations best practice is to package code as container and push to ECR using CI/CD pipelines and IaC

In this tutorial can will the Amazon Bedrock AgentCode Python SDK to easily package your artifacts and deploy them to AgentCore runtime.

### Creating runtime role

Before starting, let's create an IAM role for our AgentCore Runtime. We will do so using the utils function pre-developed for you.

In [ ]:
import sys
import os

# Get the current notebook's directory
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.join(utils_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# Add to sys.path
sys.path.insert(0, utils_dir)
print("sys.path[0]:", sys.path[0])

from utils import create_agentcore_role

agent_name="strandlanggraph_bedrock"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

sys.path[0]: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials
attaching role policy agentcore-langgraph_bedrock-role


### Configure AgentCore Runtime deployment

Next we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we just created and a requirements file. We will also configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [18]:
from bedrock_agentcore_starter_toolkit import Runtime
agentcore_runtime = Runtime()

In [15]:
agentcore_runtime.configure?

Signature:
agentcore_runtime.configure(
    entrypoint: str,
    execution_role: Optional[str] = None,
    agent_name: Optional[str] = None,
    requirements: Optional[List[str]] = None,
    requirements_file: Optional[str] = None,
    ecr_repository: Optional[str] = None,
    container_runtime: Optional[str] = None,
    auto_create_ecr: bool = True,
    auto_create_execution_role: bool = False,
    authorizer_configuration: Optional[Dict[str, Any]] = None,
    region: Optional[str] = None,
    protocol: Optional[Literal['HTTP', 'MCP']] = None,
) -> bedrock_agentcore_starter_toolkit.operations.runtime.models.ConfigureResult
Docstring:
Configure Bedrock AgentCore from notebook using an entrypoint file.

Args:
    entrypoint: Path to Python file with optional Bedrock AgentCore name
        (e.g., "handler.py" or "handler.py:bedrock_agentcore")
    execution_role: AWS IAM execution role ARN or name (optional if auto_create_execution_role=True)
    agent_name: name of the agent
    require

In [31]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="langgraph_bedrock.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    #requirements_file="requirements.txt",
    region=region
)
response

Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/langgraph_bedrock.py, bedrock_agentcore_name=langgraph_bedrock
INFO:bedrock_agentcore_starter_toolkit.utils.runtime.entrypoint:Entrypoint parsed: file=/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/langgraph_bedrock.py, bedrock_agentcore_name=langgraph_bedrock
Configuring BedrockAgentCore agent: langgraph_bedrock
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Configuring BedrockAgentCore agent: langgraph_bedrock


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/Dockerfile
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Generated Dockerfile: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/Dockerfile
Generated .dockerignore: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/.dockerignore
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.configure:Generated .dockerignore: /home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/.dockerignore
Setting 'langgraph_bedrock' as default agent
INFO:bedrock_ag

ConfigureResult(config_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/Dockerfile'), dockerignore_path=PosixPath('/home/ubuntu/projects/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/13_agentcore/tutorials/01-AgentCore-runtime/01-hosting-agent/05-strands-langgraph-with-bedrock-model/.dockerignore'), runtime='Docker', region='us-west-2', account_id='615299776985', execution_role='arn:aws:iam::615299776985:role/agentcore-langgraph_bedrock-role', ecr_repository=None, auto_create_ecr=True)

### Launching agent to AgentCore Runtime

Now that we've got a docker file, let's launch the agent to the AgentCore Runtime. This will create the Amazon ECR repository and the AgentCore Runtime

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [32]:
launch_result = agentcore_runtime.launch()

Launching Bedrock AgentCore agent 'langgraph_bedrock' to cloud
INFO:bedrock_agentcore_starter_toolkit.operations.runtime.launch:Launching Bedrock AgentCore agent 'langgraph_bedrock' to cloud


Build: exec /bin/sh: exec format error
ERROR:bedrock_agentcore_starter_toolkit.utils.runtime.container:Build: exec /bin/sh: exec format error


RuntimeError: Build failed:  ---> Using cache  ---> 488fbf9505ab Step 3/14 : COPY requirements.txt requirements.txt  ---> Using cache  ---> cd79e53fa466 Step 4/14 : RUN pip install -r requirements.txt  ---> [Warning] The requested image's platform (linux/arm64/v8) does not match the detected host platform (linux/amd64/v4) and no specific platform was requested  ---> Running in 3dad1aead37d [91mexec /bin/sh: exec format error [0mThe command '/bin/sh -c pip install -r requirements.txt' returned a non-zero code: 255

In [34]:
import platform

print(f"Machine: {platform.machine()}")
print(f"Processor: {platform.processor()}")
print(f"Architecture: {platform.architecture()}")
print(f"Platform: {platform.platform()}")

Machine: x86_64
Processor: x86_64
Architecture: ('64bit', 'ELF')
Platform: Linux-6.8.0-1008-aws-x86_64-with-glibc2.39


### Checking for the AgentCore Runtime Status
Now that we've deployed the AgentCore Runtime, let's check for it's deployment status

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How much is 2+2?"})
invoke_response

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is the weather now?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result.ecr_uri, launch_result.agent_id, launch_result.ecr_uri.split('/')[1]

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)
policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!